In [1]:
import random, pandas as pd
from datetime import datetime as dt

In [2]:
data_folder = 'G:/Geoff/Data/housing-production'
data_file = 'merged.csv'
save_file = 'cleaned_features.csv'

## Load the data and inspect its shape

In [3]:
# load sales+assessor data
dtypes = {'ucb_geo_id' : str,
          'sr_date_transfer' : str}
df_all = pd.read_csv('{}/{}'.format(data_folder, data_file), low_memory=False, dtype=dtypes)
df_all.shape

(2147507, 21)

In [4]:
df_all.head()

,sr_property_id,SA_PROPERTY_ID,MM_MUNI_NAME,MM_FIPS_MUNI_CODE,SA_SITE_MAIL_SAME,USE_CODE_STD,SA_ZONING,SA_ARCHITECTURE_CODE,SA_BLDG_SQFT,SA_CONSTRUCTION_CODE,...,SA_NBR_BEDRMS,SA_NBR_UNITS,SA_YR_BLT,SA_YR_BLT_EFFECT,sr_date_transfer,ucb_geo_id,ucb_price_sqft,ucb_price_sqft_adj,ucb_condo_subdiv_flag,ucb_condo_subdiv_sqft
0,38443515,38443515,ALAMEDA,1,Y,RSFR,NaN,NaN,1047.0,4.0,...,3.0,1.0,1952.0,1953.0,19880104,06001437400,112,206,0,1047
1,38275729,38275729,ALAMEDA,1,N,RMFD,NaN,NaN,11880.0,4.0,...,18.0,12.0,1965.0,1965.0,19880104,06001421700,37,68,0,11880
2,38264302,38264302,ALAMEDA,1,Y,RSFR,NaN,NaN,2584.0,4.0,...,5.0,1.0,1908.0,1920.0,19880104,06001423700,152,281,0,2584
3,38377769,38377769,ALAMEDA,1,N,RCON,NaN,NaN,1383.0,4.0,...,3.0,183.0,1987.0,1987.0,19880104,06001444400,90,167,0,1383
4,38490265,38490265,ALAMEDA,1,Y,RSFR,NaN,NaN,3499.0,4.0,...,4.0,1.0,1996.0,1997.0,19880104,06001443103,56,104,0,3499


In [5]:
df_all.iloc[0]

sr_property_id              38443515
SA_PROPERTY_ID              38443515
MM_MUNI_NAME                 ALAMEDA
MM_FIPS_MUNI_CODE                  1
SA_SITE_MAIL_SAME                  Y
USE_CODE_STD                    RSFR
SA_ZONING                        NaN
SA_ARCHITECTURE_CODE             NaN
SA_BLDG_SQFT                    1047
SA_CONSTRUCTION_CODE               4
SA_NBR_BATH_DQ                     0
SA_NBR_BEDRMS                      3
SA_NBR_UNITS                       1
SA_YR_BLT                       1952
SA_YR_BLT_EFFECT                1953
sr_date_transfer            19880104
ucb_geo_id               06001437400
ucb_price_sqft                   112
ucb_price_sqft_adj               206
ucb_condo_subdiv_flag              0
ucb_condo_subdiv_sqft           1047
Name: 0, dtype: object

In [6]:
# what are the first and last sale dates?
print(df_all['sr_date_transfer'].min())
print(df_all['sr_date_transfer'].min())

19880101
19880101


## Divide data into before/after

Let dummy variable `after` represent if observation at year1 instead of year0.

In [7]:
# define which years we will look at sales records for, to divide data into before/after
sale_year = 'sr_date_transfer_year'
year0 = 2009
year1 = 2014

In [8]:
# parse date string to datetime object and extract year
# have to do this instead of nice fast pd.to_datetime() style because we 
# have to handle errors from fields with bad date data
def extract_date(date, format='%Y%m%d'):
    try:
        #return pd.to_datetime(date, format=format)
        return dt.strptime(date, format)
    except:
        print('cannot convert "{}"'.format(date))
        
df_all['sr_date_transfer'] = df_all['sr_date_transfer'].map(extract_date)
df_all['sr_date_transfer_year'] = df_all['sr_date_transfer'].map(lambda x: x.year)
df_all.drop(labels='sr_date_transfer', axis='columns', inplace=True)

cannot convert "19909222"
cannot convert "19909222"
cannot convert "19909222"
cannot convert "19909222"


In [9]:
df = pd.DataFrame(df_all[df_all[sale_year].isin([year0, year1])])
df[sale_year] = df[sale_year].astype(int)
df['after'] = (df[sale_year]==year1).astype(int)

In [10]:
# how many observations do we have at each year in each county?
df.groupby(['MM_MUNI_NAME',sale_year])['ucb_price_sqft_adj'].count().unstack()

sr_date_transfer_year,2009,2014
MM_MUNI_NAME,,
ALAMEDA,15341,5034
CONTRA COSTA,16968,4707
MARIN,1597,827
NAPA,1172,422
SAN FRANCISCO,4398,1803
SAN MATEO,4718,1728
SANTA CLARA,15698,5352
SOLANO,7994,2022
SONOMA,4165,1457


In [11]:
# how much did mean sales prices change in each county from year0 to year1?
diff = df.groupby(['MM_MUNI_NAME',sale_year])['ucb_price_sqft_adj'].mean().unstack().round(2)
diff['pct_change'] = (diff[year1] / diff[year0])
diff

sr_date_transfer_year,2009,2014,pct_change
MM_MUNI_NAME,,,
ALAMEDA,244.92,338.64,1.382656
CONTRA COSTA,168.29,250.55,1.488799
MARIN,382.41,473.72,1.238775
NAPA,223.66,293.31,1.311410
SAN FRANCISCO,529.85,696.92,1.315316
SAN MATEO,424.71,547.78,1.289774
SANTA CLARA,327.96,447.39,1.364160
SOLANO,128.04,163.38,1.276007
SONOMA,216.12,270.16,1.250046


In [12]:
# how many rows in data set now?
len(df)

95403

## Divide data into treatment/control

Assign to treatment group if property had at least *n* new housing units built within *m* meters of it between year0 and year1.

In [13]:
# proximity to new building construction


In [14]:
# until we have housing units built var, just assign to groups randomly
df['treat'] = df.index.map(lambda x: random.randint(0, 1))

# or say that 'single family residence' is the treatment group
df['treat'] = (df['USE_CODE_STD']=='RSFR').astype(int)

## Feature engineering

In [15]:
# how old was the building at year1
df['age'] = year1 - df['SA_YR_BLT']

In [16]:
# convert SA_SITE_MAIL_SAME to ternary variable
# to make it a binary dummy, we'll have to handle the unknown "U" cases by dropping them
mapping = {'N' : 0,
           'Y' : 1,
           'U' : 2}
df['SA_SITE_MAIL_SAME'] = df['SA_SITE_MAIL_SAME'].replace(mapping)
mask = ~(df['SA_SITE_MAIL_SAME']==2)
df = df.loc[mask]

In [17]:
# dummy variable for if sale was after construction (1 if yes, 0 if no)
df['sale_after_construction'] = df['sr_date_transfer_year'] > df['SA_YR_BLT']
df['sale_after_construction'] = df['sale_after_construction'].astype(int)

In [18]:
# adjust bathrooms and bedrooms by the number of units?
# Sam created a flag for condo subdivision in the sales dataset that may help with the units issue from the assessor
# but handle zero units to not get inf values
bedrooms_per_unit = df['SA_NBR_BEDRMS'] / df['SA_NBR_UNITS']
bedrooms_per_unit.head()

354186         inf
354321    2.000000
355893    3.000000
356237    4.000000
356270    3.000000
dtype: float64

In [19]:
# retain only certain use codes, and convert to dummies


In [20]:
# identify subsidized units (and drop them?)


In [21]:
# Other independent variables we'll add in later are at the neighborhood level, which we can merge in with the fips numbers.


In [22]:
df.iloc[0]

sr_property_id                38359698
SA_PROPERTY_ID                38359698
MM_MUNI_NAME                   ALAMEDA
MM_FIPS_MUNI_CODE                    1
SA_SITE_MAIL_SAME                    0
USE_CODE_STD                      RSFR
SA_ZONING                          NaN
SA_ARCHITECTURE_CODE               NaN
SA_BLDG_SQFT                       570
SA_CONSTRUCTION_CODE                 4
SA_NBR_BATH_DQ                       0
SA_NBR_BEDRMS                        1
SA_NBR_UNITS                         0
SA_YR_BLT                         1934
SA_YR_BLT_EFFECT                  1934
ucb_geo_id                 06001430102
ucb_price_sqft                     821
ucb_price_sqft_adj                 837
ucb_condo_subdiv_flag                0
ucb_condo_subdiv_sqft              570
sr_date_transfer_year             2009
after                                0
treat                                1
age                                 80
sale_after_construction              1
Name: 354186, dtype: obje

In [23]:
df.to_csv('{}/{}'.format(data_folder, save_file), index=False, encoding='utf-8')

## Exploring...

In [24]:
mask_0units = df['SA_NBR_UNITS']==0
df[mask_0units]['USE_CODE_STD'].value_counts()

RCON    6629
RSFR    6008
RMFD     284
RMSC     156
RAPT     119
RDUP      30
RCOO      11
RMOB       4
RTRI       3
RQUA       2
Name: USE_CODE_STD, dtype: int64

DataQuick Residential Use Codes

```
RAPT	Multi-Family Res (5+ Units)
RCON	Condominium, PUD
RCOO	Cooperative
RDUP	Duplex
RMFD	Multi-Family Dwelling (2-4 Units)
RMOB	Mobile / Manufactured Home
RMSC	Miscellaneous Residential
RQUA	Quadraplex
RSFR	Single Family Residence
RTIM	Timeshare
RTRI	Triplex
```

In [25]:
df[df['USE_CODE_STD']=='RSFR']['SA_NBR_UNITS'].value_counts().sort_index().tail()

3100.0    1
5800.0    1
6400.0    1
6500.0    1
7300.0    1
Name: SA_NBR_UNITS, dtype: int64

In [26]:
counts = df.groupby(['MM_MUNI_NAME', 'USE_CODE_STD'])['sr_property_id'].count()

In [27]:
df_0br = df[df['SA_NBR_BEDRMS'] <= 0]
df_0ba = df[df['SA_NBR_BATH_DQ'] <= 0]
df_0units = df[df['SA_NBR_UNITS'] <= 0]

In [28]:
counts_0br = df_0br.groupby(['MM_MUNI_NAME', 'USE_CODE_STD'])['sr_property_id'].count()
counts_0ba = df_0ba.groupby(['MM_MUNI_NAME', 'USE_CODE_STD'])['sr_property_id'].count()
counts_0units = df_0units.groupby(['MM_MUNI_NAME', 'USE_CODE_STD'])['sr_property_id'].count()

In [29]:
df_counts = pd.DataFrame({'count_observations':counts,
                          'with_0br':counts_0br/counts,
                          'with_0ba':counts_0ba/counts,
                          'with_0units':counts_0units/counts}).fillna(0)

In [30]:
df_counts.to_csv('count-zeroes.csv', index=True, encoding='utf-8')

In [31]:
df[(df['USE_CODE_STD']=='RSFR') & (df['SA_NBR_UNITS']>=5000)].sort_values('SA_NBR_UNITS')

,sr_property_id,SA_PROPERTY_ID,MM_MUNI_NAME,MM_FIPS_MUNI_CODE,SA_SITE_MAIL_SAME,USE_CODE_STD,SA_ZONING,SA_ARCHITECTURE_CODE,SA_BLDG_SQFT,SA_CONSTRUCTION_CODE,...,ucb_geo_id,ucb_price_sqft,ucb_price_sqft_adj,ucb_condo_subdiv_flag,ucb_condo_subdiv_sqft,sr_date_transfer_year,after,treat,age,sale_after_construction
1750165,30567769,30567769,SANTA CLARA,85,1,RSFR,R1-8,NaN,1137.0,4.0,...,06085503208,242,248,0,1137,2009,0,1,56.0,1
1758721,30539102,30539102,SANTA CLARA,85,0,RSFR,R1-8,NaN,1484.0,4.0,...,06085503322,256,261,0,1484,2009,0,1,49.0,1
1761517,30539067,30539067,SANTA CLARA,85,0,RSFR,R1-8,NaN,1564.0,4.0,...,06085503322,223,228,0,1564,2009,0,1,49.0,1
1825646,30437231,30437231,SANTA CLARA,85,1,RSFR,R1-8,NaN,1652.0,4.0,...,06085502908,544,501,0,1652,2014,1,1,41.0,1


In [32]:
# why are there many RSFRs with 5,000+ units?
# drop all mobile and timeshare?

In [33]:
multi_unit_uses = ['RAPT', 'RCON', 'RCOO', 'RDUP', 'RMFD', 'RMSC', 'RQUA', 'RTRI']
mask_multi_unit = df['USE_CODE_STD'].isin(multi_unit_uses)

In [34]:
df_filtered = df[~(mask_0units & mask_multi_unit)]

In [35]:
print(len(df))
print(len(df_filtered))

95209
87975


In [36]:
df[mask_0units].iloc[:5].T

,354186,379878,379879,379884,379885
sr_property_id,38359698,119169079,113580407,122800907,109701857
SA_PROPERTY_ID,38359698,119169079,113580407,122800907,109701857
MM_MUNI_NAME,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA,ALAMEDA
MM_FIPS_MUNI_CODE,1,1,1,1,1
SA_SITE_MAIL_SAME,0,0,1,1,1
USE_CODE_STD,RSFR,RCON,RSFR,RSFR,RCON
SA_ZONING,NaN,NaN,NaN,NaN,NaN
SA_ARCHITECTURE_CODE,NaN,NaN,NaN,NaN,1
SA_BLDG_SQFT,570,1741,3774,1400,693
SA_CONSTRUCTION_CODE,4,4,4,4,4
